In [1]:
# from google.colab import drive
# import os
# import sys
# drive.mount('/content/drive')
# os.chdir("./drive/MyDrive/Project/")


Mounted at /content/drive


In [2]:
%%capture
!pip install np_utils
!pip install transformers
!pip install pickle5
!pip3 install keras-visualizer


In [3]:
import numpy as np
import pandas as pd
import librosa
import pickle
import re
import pickle5 as pickle

from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import  accuracy_score,f1_score 

import tensorflow as tf
import tensorflow_datasets as tfds

from transformers import RobertaTokenizer, RobertaModel, TFRobertaForSequenceClassification

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')


In [4]:
%%capture
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')


In [5]:
def data_format(data):
    label = []
    col = list(test.columns)[2:] # get emotions
    for index, row in data.iterrows():
        lab = []
        for i in col: 
            lab.append(row[i])
        label.append(lab)
    return label 


### Load Data


In [6]:
test = pd.read_csv("./BertData/2018-E-c-En-train.txt",  sep="\t")
train = pd.read_csv("./BertData/2018-E-c-En-test-gold.txt", sep="\t")
validation = pd.read_csv("./BertData/2018-E-c-En-dev.txt", sep="\t")
train = train.append(validation, ignore_index=True)
train = train.append(test, ignore_index=True)

s = pd.Series(list(range(len(train))))
train=train.set_index(s)

reset = train
train_label = data_format(train) 

label_reset = train_label



10983


In [7]:
max_length = 128
batch_size = 32


def map_example_to_dict(input_ids, attention_masks, label):
    # print(label[0])
    return {
      "input_ids": input_ids,
      "attention_mask": attention_masks,
           }, label[0]
def encode_examples(ds, limit=-1):
  input_id = []
  attention_mask = []
  labels = []
  for one_data, label in tfds.as_numpy(ds):
    bert_input = tokenizer.encode_plus(one_data.decode(),add_special_tokens=True,max_length=max_length,pad_to_max_length=True,return_attention_mask=True,)
    input_id.append(bert_input['input_ids'])
    attention_mask.append(bert_input['attention_mask'])
    labels.append([label])

  return tf.data.Dataset.from_tensor_slices((input_id,
                                             attention_mask,
                              labels)).map(map_example_to_dict)

In [12]:
learning_rate = 2e-5
number_of_epochs = 7

class ModelMetrics(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.count_n = 1
        self.best = 0

    def on_epoch_end(self, batch, logs={}):


        trm  = self.model.predict(ds_train_encoded)['logits']
        y_val_pred_train = tf.nn.softmax(trm)
        y_pred_argmax_train = tf.math.argmax(y_val_pred_train, axis=1)

        y_val_pred = tf.nn.softmax(self.model.predict(ds_test_encoded)['logits'])
        y_pred_argmax = tf.math.argmax(y_val_pred, axis=1)
        training_copy = {}
        training_copy['predicted'] = y_pred_argmax_train 

        testing_copy = {}
        testing_copy['predicted'] = y_pred_argmax 
        # micro = f1_score(test_label, testing_copy['predicted'].numpy(), average='micro')
        # weight = f1_score(test_label, testing_copy['predicted'].numpy(), average='weighted')
        # print('\n accuracy score is :', f1_s, micro, weight)

        pred = testing_copy['predicted'].numpy()
        cnt = 0 
        total = len(test_label)

        self.count_n += 1





In [13]:
KF = KFold(n_splits=5, random_state=42, shuffle=True)
for train_index, test_index in KF.split(reset):
    train = reset
    K_label = label_reset
    test = train.loc[test_index, :]
    train = train.loc[train_index, :]
    train_label = [K_label[i] for i in train_index]
    test_label = [K_label[i] for i in test_index]

    training_sentences_modified = tf.data.Dataset.from_tensor_slices((train['Tweet'],
                                                                    train_label)) # train_label train['label']
    testing_sentences_modified = tf.data.Dataset.from_tensor_slices((test['Tweet'],
                                                                    test_label)) # test_label test['label']

    ds_train_encoded = encode_examples(training_sentences_modified).shuffle(10000).batch(batch_size)
    ds_test_encoded = encode_examples(testing_sentences_modified).batch(batch_size)
    metrics = ModelMetrics()

    model = TFRobertaForSequenceClassification.from_pretrained("roberta-base", num_labels = 11) #len(train['label'].unique())
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-05)
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

    model.compile(optimizer=optimizer, loss=loss,metrics=['categorical_accuracy','accuracy',tf.keras.metrics.BinaryAccuracy()]) # , metrics=[metric]

    model.fit(ds_train_encoded, epochs=number_of_epochs,
            validation_data=ds_test_encoded, callbacks=[metrics])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/7
275/275 [==============================] - 156s 517ms/step - loss: 0.4370 - categorical_accuracy: 0.3742 - accuracy: 0.3742 - binary_accuracy: 0.8157 - val_loss: 0.3365 - val_categorical_accuracy: 0.5753 - val_accuracy: 0.5753 - val_binary_accuracy: 0.8585
Epoch 2/7
 39/275 [===>..........................] - ETA: 1:50 - loss: 0.3239 - categorical_accuracy: 0.5721 - accuracy: 0.5721 - binary_accuracy: 0.8655

KeyboardInterrupt: ignored